# Kinase fold structural alignment

This notebook showcases alignment of kinases by the structure of their ATP-binding folds.

The user may select any two human kinases and their ATP-binding domains will be displayed with structures aligned and superimposed. Alignment is done by the TM-align algorithm.

For this visualization, we use high-confidence predicted structures from AlphaFold2 (https://alphafold.ebi.ac.uk/).

## Instructions

**Choose any two human kinases below by entering their Uniprot entry names.**

You may choose to display only the kinase domain (default) or the full proteins.

You may also choose the colors to display the proteins.

In [14]:
### choose proteins to align
### Note: for kinases with multiple ATP-binding domains, append "_1", "_2", etc. to align the specified domain
prot1 = "STK11_HUMAN"
prot2 = "GRK7_HUMAN"
### choose whether to use full proteins or pockets only
full_proteins = True
### choose colors for proteins
col1 = "cyan"
col2 = "yellow"

### Other examples

In [15]:
#prot1 = "PK3CA_HUMAN"
#prot2 = "PK3CB_HUMAN"
#prot1 = "MTOR_HUMAN"
#prot2 = "ATR_HUMAN"
#prot1 = "AKT1_HUMAN"
#prot2 = "AKT2_HUMAN"
#prot1 = "KS6A6_HUMAN_1" ### only works with full_proteins == False
#prot2 = "KS6A6_HUMAN_2" ### only works with full_proteins == False

## Main code

### Install necessary Python packages

In [1]:
!pip install biopython
!pip install pyprojroot
!pip install py3Dmol

     |████████████████████████████████| 2.3 MB 26.0 MB/s 
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
  File 

### Load packages

In [ ]:
from Bio.PDB import *
from pyprojroot import here
import os
#import nglview as nv
import py3Dmol
import numpy as np
import pickle
import pandas as pd
import seaborn as sns

### Download data

In [ ]:
pf = "AF2_pocketome_tm_score.pkl"
check = not os.path.exists(pf)
call = "wget " + "https://github.com/NicholasClark/collab_kinase_proj/raw/master/" + pf
print(call)
if check:
  os.system(call)

### Load necessary data files -- a pkl file of TM-align output and a CSV with kinase metadata

In [ ]:
### load pkl file
pkl_file = "AF2_pocketome_tm_score.pkl"
f = open(pkl_file, "rb")
tmp = pickle.load(f)
f.close()

### load csv file with metadata
csv_file = "https://github.com/NicholasClark/collab_kinase_proj/raw/master/kinome_plus_pocket_meta_2022_04_06.csv"
df = pd.read_csv(csv_file)

### Kinase metadata

In [ ]:
### the column "uniprot_name_mod" contains uniprot entry names for all human kinases
### Note: kinases with multiple ATP-binding domains are listed multiple times here, with "_1", "_2", etc. appended.
df[['uniprot_name_mod', 'uniprot_accession', 'gene_names', 'protein_names', 'hgnc_symbol','is_idg_dark_kinase']]

### Helper functions

In [ ]:
## function to take uniprot name and translate to the accession number
## @input uni_name - a uniprot name (i.e. "STK11_HUMAN") -- kinases with multiple ATP-binding domains will have "_1" or "_2" appended 
## @return the corresponding uniprot accession number -- again some will be appended with "_1" or "_2"
def uni_name_to_acc(uni_name):
    ind = np.where(df.uniprot_name_mod == uni_name)
    uni_acc = df.uniprot_accession_mod[int(ind[0])]
    return(uni_acc)

## function to take two proteins and get the TM-align rotation matrix + translation vector
## @input a,b - two uniprot accession numbers 
## @return u,t - a rotation matrix u and a translation vector t
def get_rot_mat(a,b):
    l1 = tmp['uni1']
    l2 = tmp['uni2']
    u_list = tmp['u']
    t_list = tmp['t']
    tm1 = tmp['tm1']
    tm2 = tmp['tm2']
    for i in range(0,len(l1)):
        if l1[i] == a and l2[i] == b:
            u = u_list[i]
            t = t_list[i]
            tm_1 = tm1[i]
            tm_2 = tm2[i]
            order = "same"
            return(u, t, tm_1, tm_2, order)
        if l1[i] == b and l2[i] == a:
            u = u_list[i]
            t = t_list[i]
            tm_1 = tm1[i]
            tm_2 = tm2[i]
            order = "reverse"
            return(u, t, tm_1, tm_2, order)
        ## if not found, return empty lists
    return([], [], [], [], [])

In [ ]:
acc1 = uni_name_to_acc(prot1)
acc2 = uni_name_to_acc(prot2)
file1 = "https://github.com/NicholasClark/collab_kinase_proj/raw/master/AF2_kinase_pockets/" + acc1 + "_pocket_only.pdb"
file2 = "https://github.com/NicholasClark/collab_kinase_proj/raw/master/AF2_kinase_pockets/" + acc2 + "_pocket_only.pdb"
os.system("wget " + file1)
os.system("wget " + file2)

### Load and align kinase structures

In [ ]:
acc1 = uni_name_to_acc(prot1)
acc2 = uni_name_to_acc(prot2)

#print(prot1 + ": " + acc1)
#print(prot2 + ": " + acc2)

if full_proteins:
    file1 = "AF-" + acc1 + "-F1-model_v2.pdb"
    file2 = "AF-" + acc2 + "-F1-model_v2.pdb"
    os.system("wget " + "https://github.com/NicholasClark/collab_kinase_proj/raw/master/AF2/" + file1)
    os.system("wget " + "https://github.com/NicholasClark/collab_kinase_proj/raw/master/AF2/" + file2)
else:
    file1 = acc1 + "_pocket_only.pdb"
    file2 = acc2 + "_pocket_only.pdb"
    os.system("wget " + "https://github.com/NicholasClark/collab_kinase_proj/raw/master/AF2_kinase_pockets/" + file1)
    os.system("wget " + "https://github.com/NicholasClark/collab_kinase_proj/raw/master/AF2_kinase_pockets/" + file2)

In [ ]:
parser = PDBParser()
str1 = parser.get_structure(prot1, file1)
str1_orig = parser.get_structure(prot1, file1)
str2 = parser.get_structure(prot2, file2)
str2_orig = parser.get_structure(prot2, file2)

u,t,tm1,tm2,order = get_rot_mat(acc1, acc2)

if order == "same":
    str1.transform(u.T, t)
else:
    str2.transform(u.T, t)

tmp_f1 = "str1.pdb"
tmp_f2 = "str2.pdb"
io=PDBIO()
io.set_structure(str1)
io.save(tmp_f1)
io=PDBIO()
io.set_structure(str2)
io.save(tmp_f2)

## Output

### Kinase alignment visualization

The aligned kinases are displayed along with their TM-scores.
Note that the TM-score is normalized by the length of the reference protein, so there are two possible TM-scores -- both are displayed:

In [ ]:
### print TM-score
print(prot1 + ": " + acc1)
print(prot2 + ": " + acc2)
print("TM-score 1: " + str( round(tm1, 2)) )
print("TM-score 2: " + str( round(tm2, 2)) )
### View protein overlays
view = py3Dmol.view()
view.addModel(open(tmp_f1, 'r').read(), 'pdb')
view.setStyle({'model': 0},{'cartoon':{'color':col1}})
view.addModel(open(tmp_f2, 'r').read(), 'pdb')
view.setStyle({'model': 1},{'cartoon':{'color':col2}})
view.zoomTo()
view.show()

### Heatmap of TM-scores

This heatmap shows pairwise TM-scores of kinases.
Each TM-score is between 0 (no alignment) and 1 (perfect alignment) and 

In [ ]:
mat = tmp['tm_max_mat']
sns.clustermap(mat, cmap="YlOrRd", vmin=0, vmax=1)